# Moving into a Toronto neighborhood

In order to create the dataframe, I copied the data to an Excel file (postal_canada.xls), removed all the unnecessary data and read the file using pandas.

In order to add the Latitude and Longitude to the dataframe, I used the latitudes and longitudes on the Geospatial_Coordinates.csv file and added the coordinates to my previous Excel file (postal_canada.xls), creating a new Excel file (postal_canada_latlong.xls) with all the information.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_latlong = pd.read_excel('postal_canada_latlong.xls')
df_latlong.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [42]:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium=0.5.0 --yes

In [3]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Find the coordinates for Toronto:

In [4]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


Create map of Toronto using latitude and longitude values: 

In [5]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighborhood in zip(df_latlong['Latitude'], df_latlong['Longitude'], df_latlong['Borough'], df_latlong['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

In [6]:
#Defining the Foursquare API credentials
CLIENT_ID = 'MRWY4MHSKYXXLFEKFTXD3YNMWF3NDVC4H3IGP4TCXESFF0A1'
CLIENT_SECRET = 'I2YK3NDB5CD1EVQLQVFO4YJHYZNYZIU4JYA0A1PCVF4FKOR1'
VERSION = '20180605' # Foursquare API versionS

The following function will be used to get the nearby venues:

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create new dataframe with the venues:

In [8]:
LIMIT = 100

toronto_venues = getNearbyVenues(names=df_latlong['Neighborhood'],
                                   latitudes=df_latlong['Latitude'],
                                   longitudes=df_latlong['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

Examine each neighbourhood and their venues:

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
print(toronto_onehot.shape)
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

(2121, 270)


,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.029412,0.0,0.0,0.0,0.0
90,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.111111,0.000000,0.0,0.0,0.0,0.0


In [28]:
toronto_grouped.drop([ 'Accessories Store', 'Afghan Restaurant', 'American Restaurant', 'Antique Shop',  'Auto Garage', 'Auto Workshop', 'BBQ Joint',  'Bagel Shop', 'Bakery', 'Bank',   'Baseball Field', 'Baseball Stadium', 'Basketball Court', 'Basketball Stadium',  'Beach',  'Beer Bar', 'Beer Store', 'Belgian Restaurant', 'Bike Shop', 'Bistro',  'Board Shop', 'Boat or Ferry', 'Bookstore', 'Art Museum',  'Boutique', 'Brazilian Restaurant', 'Breakfast Spot', 'Brewery', 'Bridal Shop', 'Bubble Tea Shop',  'Building', 'Burger Joint', 'Burrito Place', 'Bus Line', 'Bus Station', 'Bus Stop', 'Business Service', 'Butcher',  'Cajun / Creole Restaurant', 'Camera Store',  'Candy Store', 'Caribbean Restaurant', 'Cheese Shop', 'Chinese Restaurant', 'Chocolate Shop', 'Church', 'Climbing Gym', 'Clothing Store',  'Cocktail Bar',  'Coffee Shop',  'College Auditorium',  'College Rec Center', 'College Stadium', 'Colombian Restaurant', 'Comfort Food Restaurant', 'Comic Shop',   'Construction & Landscaping', 'Bed & Breakfast', 'Art Museum', 'Athletics & Sports',  'Bookstore', 'Cosmetics Shop',  'Creperie', 'Cuban Restaurant',  'Cupcake Shop', 'Curling Ice',  'Dance Studio', 'Deli / Bodega',  'Department Store',  'Dessert Shop', 'Dim Sum Restaurant',  'Diner', 'Discount Store',  'Distribution Center',  'Dog Run', 'Doner Restaurant',  'Donut Shop', 'Drugstore',  'Dumpling Restaurant',  'Eastern European Restaurant',  'Electronics Store',  'Ethiopian Restaurant',  'Event Space',  'Falafel Restaurant',  'Farmers Market',  'Fast Food Restaurant',  'Field',  'Filipino Restaurant',  'Fish & Chips Shop',  'Fish Market',  'Flea Market',  'Food & Drink Shop',  'Food Court',  'Food Truck',  'Fountain',  'French Restaurant',  'Fried Chicken Joint',  'Frozen Yogurt Shop',  'Fruit & Vegetable Store',  'Furniture / Home Store',  'Gaming Cafe',  'Garden',  'Garden Center',  'Gas Station',  'Gastropub',  'Gay Bar',  'General Entertainment',  'General Travel',  'German Restaurant',  'Gift Shop',  'Gluten-free Restaurant',  'Golf Course',  'Gourmet Shop',  'Greek Restaurant',  'Grocery Store',  'Gym',  'Gym / Fitness Center',  'Hakka Restaurant',  'Harbor / Marina',  'Hardware Store',  'Health & Beauty Service',  'Health Food Store',  'Historic Site',  'History Museum',  'Hobby Shop',  'Hockey Arena',  'Home Service',  'Hookah Bar',  'Hospital',  'Hostel',   'Hotel Bar',  'IT Services',  'Ice Cream Shop',  'Indian Restaurant',  'Indie Movie Theater',  'Indonesian Restaurant',  'Intersection',  'Irish Pub',  'Italian Restaurant',  'Japanese Restaurant',  'Jazz Club',  'Jewelry Store',  'Juice Bar',  'Korean Restaurant',  'Lake',  'Latin American Restaurant',  'Light Rail Station',  'Lingerie Store',  'Liquor Store',  'Lounge',  'Luggage Store',  'Market',  'Martial Arts School',  'Massage Studio',  'Medical Center',  'Mediterranean Restaurant',  "Men's Store",  'Metro Station',  'Mexican Restaurant',  'Middle Eastern Restaurant',  'Miscellaneous Shop',  'Mobile Phone Shop',  'Modern European Restaurant',  'Molecular Gastronomy Restaurant',  'Monument / Landmark',  'Moroccan Restaurant',  'Motel',  'Movie Theater',  'Museum',  'Music Venue',  'New American Restaurant',  'Nightclub',  'Noodle House',  'Office',  'Opera House',  'Optical Shop',  'Organic Grocery',  'Other Great Outdoors',  'Performing Arts Venue',  'Pharmacy',  'Pizza Place',  'Plane',  'Playground',  'Plaza',  'Poke Place',  'Pool',  'Portuguese Restaurant',  'Poutine Place',  'Ramen Restaurant',  'Record Shop',  'Recording Studio',  'Rental Car Location',  'Restaurant',  'River',  'Roof Deck',  'Sake Bar',  'Salad Place',  'Salon / Barbershop',  'Sandwich Place',  'Scenic Lookout',  'Sculpture Garden',  'Seafood Restaurant',  'Shoe Store',  'Skate Park',  'Skating Rink',  'Smoke Shop',  'Smoothie Shop',  'Snack Place',  'Soccer Field', 'Soup Place',  'Spa',  'Speakeasy',  'Sporting Goods Shop',  'Sports Bar',  'Stadium',  'Stationery Store',  'Steakhouse',  'Strip Club',  'Supplement Shop',  'Sushi Restaurant',  'Swim School',  'Taco Place',  'Tailor Shop',  'Taiwanese Restaurant',  'Tanning Salon',  'Tea Room',  'Tennis Court',  'Thai Restaurant',  'Theater',  'Theme Restaurant',  'Thrift / Vintage Store',  'Toy / Game Store',  'Trail',  'Train Station',  'Vegetarian / Vegan Restaurant',  'Video Store',  'Vietnamese Restaurant',  'Warehouse Store',  'Wine Bar', 'Wings Joint',  "Women's Store"], axis = 1, inplace=True)
toronto_grouped.shape

(94, 31)

In [29]:
toronto_grouped.drop(['Airport Food Court', 'Airport Lounge', 'Airport Service', 'Airport Terminal', 'Aquarium', 'Knitting Store', 'Moving Target', 'Outdoor Sculpture'], axis = 1, inplace=True)

In [30]:
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Art Gallery,Arts & Crafts Store,Asian Restaurant,Baby Store,Bar,Café,College Arts Building,...,Coworking Space,Hotel,Kids Store,Park,Pet Store,Pub,Rental Service,Shopping Mall,Supermarket,Video Game Store
0,Agincourt,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.142857,0.0,0.000000,0.000000,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.047619,0.000000,0.000000,0.0,0.047619,0.047619,0.0
3,Bayview Village,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.250000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.037037,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.037037,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,"Willowdale, Willowdale East",0.0,0.0,0.0,0.029412,0.0,0.0,0.000000,0.058824,0.0,...,0.0,0.029412,0.0,0.000000,0.029412,0.000000,0.0,0.029412,0.000000,0.0
90,"Willowdale, Willowdale West",0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
91,Woburn,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.0


Creating the dataframes for each one of the cases (family, bussiness person, young man e young woman) only with the relevent venues for them.

In [54]:
toronto_fam = toronto_grouped[['Neighborhood', 'Baby Store', 'Pet Store', 'Shopping Mall',  'Convenience Store',  'Supermarket']]
toronto_buss = toronto_grouped[['Neighborhood', 'Hotel',  'Airport', 'Café', 'Pub',  'Coworking Space']]
toronto_boy = toronto_grouped[['Neighborhood','College Gym', 'Bar',  'Video Game Store',  'Park',  'Concert Hall']]
toronto_girl = toronto_grouped[['Neighborhood',  'College Arts Building',  'Art Gallery', 'Arts & Crafts Store',  'Yoga Studio',  'Asian Restaurant']]

### The family

In [45]:
toronto_fam['Frequency'] = toronto_fam.sum(axis=1)
toronto_fam = toronto_fam[toronto_fam.Frequency > 0]
toronto_fam.shape

(29, 7)

In [48]:
toronto_fam.sort_values(by=['Frequency'], ascending= False).head()

,Neighborhood,Baby Store,Pet Store,Shopping Mall,Convenience Store,Supermarket,Frequency
87,Weston,0.0,0.000,0.0,0.500000,0.0,0.500000
93,York Mills West,0.0,0.000,0.0,0.333333,0.0,0.333333
69,"Runnymede, The Junction North",0.0,0.000,0.0,0.250000,0.0,0.250000
27,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.0,0.125,0.0,0.125000,0.0,0.250000
26,"East Toronto, Broadview North (Old East York)",0.0,0.000,0.0,0.250000,0.0,0.250000


In [49]:
toronto_fam.sort_values(by=['Baby Store'], ascending= False).head()

,Neighborhood,Baby Store,Pet Store,Shopping Mall,Convenience Store,Supermarket,Frequency
14,Christie,0.0625,0.000000,0.000000,0.0,0.000000,0.062500
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0000,0.000000,0.047619,0.0,0.047619,0.095238
41,"India Bazaar, The Beaches West",0.0000,0.050000,0.000000,0.0,0.000000,0.050000
89,"Willowdale, Willowdale East",0.0000,0.029412,0.029412,0.0,0.000000,0.058824
87,Weston,0.0000,0.000000,0.000000,0.5,0.000000,0.500000


### The business person

In [51]:
toronto_buss['Frequency'] = toronto_buss.sum(axis=1)
toronto_buss = toronto_buss[toronto_buss.Frequency > 0]
toronto_buss.sort_values(by=['Frequency'], ascending= False).head()

,Neighborhood,Hotel,Airport,Café,Pub,Coworking Space,Frequency
3,Bayview Village,0.0,0.0,0.25,0.00,0.0,0.25
32,Glencairn,0.0,0.0,0.00,0.25,0.0,0.25
6,"Birch Cliff, Cliffside West",0.0,0.0,0.25,0.00,0.0,0.25
79,The Beaches,0.0,0.0,0.00,0.25,0.0,0.25
78,"The Annex, North Midtown, Yorkville",0.0,0.0,0.15,0.05,0.0,0.20


### The young man

In [56]:
#toronto_boy['Frequency'] = toronto_boy.sum(axis=1)
toronto_boy = toronto_boy[toronto_boy.Frequency > 0]
toronto_boy.sort_values(by=['Frequency'], ascending= False).head()

,Neighborhood,College Gym,Bar,Video Game Store,Park,Concert Hall,Frequency
93,York Mills West,0.0,0.333333,0.0,0.333333,0.0,0.666667
10,Caledonia-Fairbanks,0.0,0.000000,0.0,0.500000,0.0,0.500000
26,"East Toronto, Broadview North (Old East York)",0.0,0.000000,0.0,0.500000,0.0,0.500000
61,Parkwoods,0.0,0.000000,0.0,0.500000,0.0,0.500000
67,"Rouge Hill, Port Union, Highland Creek",0.0,0.500000,0.0,0.000000,0.0,0.500000


### The young woman

In [59]:
#toronto_girl['Frequency'] = toronto_girl.sum(axis=1)
toronto_girl = toronto_girl[toronto_girl.Frequency > 0]
toronto_girl.sort_values(by=['Frequency'], ascending= False).head()

,Neighborhood,College Arts Building,Art Gallery,Arts & Crafts Store,Yoga Studio,Asian Restaurant,Frequency
48,"Little Portugal, Trinity",0.000000,0.021739,0.000000,0.021739,0.065217,0.217391
22,Don Mills,0.000000,0.041667,0.000000,0.000000,0.041667,0.166667
62,"Queen's Park, Ontario Provincial Government",0.000000,0.000000,0.030303,0.030303,0.000000,0.121212
8,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.058824,0.000000,0.117647
84,"University of Toronto, Harbord",0.028571,0.000000,0.000000,0.028571,0.000000,0.114286
